In [8]:
import json 

# load json file
with  open("reddit_tech_scrape[1].json", "r", encoding="utf-8") as file:
    data = json.load(file)



print("first 2 entries:", data[:2])
print("data type:", type(data))

first 2 entries: [{'subreddit': 'technology', 'title': 'FBI Says Backup Now—Confirms Dangerous Attacks Underway', 'author': 'lurker_bee', 'url': 'https://www.forbes.com/sites/daveywinder/2025/02/21/new-fbi-warning-backup-today-as-dangerous-attacks-ongoing/', 'upvotes': 25173, 'comments': ['WARNING! The link in question may require you to disable ad-blockers to see content.  Though not required, please consider submitting an alternative source for this story.\n\nWARNING! Disabling your ad blocker may open you up to malware infections, malicious cookies and can expose you to unwanted tracker networks.  PROCEED WITH CAUTION.\n\nDo not open any files which are automatically downloaded, and do not enter personal information on any page you do not trust. \nIf you are concerned about tracking, consider opening the page in an incognito window, and verify that your browser is sending "do not track" requests.  \n\nIF YOU ENCOUNTER ANY MALWARE, MALICIOUS TRACKERS, CLICKJACKING, OR REDIRECT LOOPS 

In [9]:
import pandas as pd
df = pd.DataFrame(data)
print("columns in dataframe:", df.columns)

columns in dataframe: Index(['subreddit', 'title', 'author', 'url', 'upvotes', 'comments'], dtype='object')


In [10]:
print(df['comments'].apply(type).value_counts())

comments
<class 'list'>    50
Name: count, dtype: int64


In [12]:
df_exploded = df.explode('comments')
df_exploded.rename(columns={"comments": "comment"}, inplace=True)
df_exploded.dropna(subset=["comment"], inplace=True)
df_exploded.reset_index(drop=True, inplace=True)
print(df_exploded.head())

    subreddit                                              title      author  \
0  technology  FBI Says Backup Now—Confirms Dangerous Attacks...  lurker_bee   
1  technology  FBI Says Backup Now—Confirms Dangerous Attacks...  lurker_bee   
2  technology  FBI Says Backup Now—Confirms Dangerous Attacks...  lurker_bee   
3  technology  FBI Says Backup Now—Confirms Dangerous Attacks...  lurker_bee   
4  technology  FBI Says Backup Now—Confirms Dangerous Attacks...  lurker_bee   

                                                 url  upvotes  \
0  https://www.forbes.com/sites/daveywinder/2025/...    25173   
1  https://www.forbes.com/sites/daveywinder/2025/...    25173   
2  https://www.forbes.com/sites/daveywinder/2025/...    25173   
3  https://www.forbes.com/sites/daveywinder/2025/...    25173   
4  https://www.forbes.com/sites/daveywinder/2025/...    25173   

                                             comment  
0  WARNING! The link in question may require you ...  
1  For emphasis: \

In [13]:
def is_valid_comment(comment):
    bot_phrases= [
        "I am a bot", "PLease contact the moderators", "WARNING", "proceed with caution", "malicious trackers"
    ]
    return not any(phrase in comment for phrase in bot_phrases)

df_exploded = df_exploded[df_exploded["comment"].apply(is_valid_comment)]
df_exploded.reset_index(drop=True, inplace=True)
print(df_exploded.head())

    subreddit                                              title      author  \
0  technology  FBI Says Backup Now—Confirms Dangerous Attacks...  lurker_bee   
1  technology  FBI Says Backup Now—Confirms Dangerous Attacks...  lurker_bee   
2  technology  FBI Says Backup Now—Confirms Dangerous Attacks...  lurker_bee   
3  technology  FBI Says Backup Now—Confirms Dangerous Attacks...  lurker_bee   
4  technology  FBI Says Backup Now—Confirms Dangerous Attacks...  lurker_bee   

                                                 url  upvotes  \
0  https://www.forbes.com/sites/daveywinder/2025/...    25173   
1  https://www.forbes.com/sites/daveywinder/2025/...    25173   
2  https://www.forbes.com/sites/daveywinder/2025/...    25173   
3  https://www.forbes.com/sites/daveywinder/2025/...    25173   
4  https://www.forbes.com/sites/daveywinder/2025/...    25173   

                                             comment  
0  For emphasis: \n\n"Ghost prefers to use public...  
1  > The Ghost ran

In [14]:
import re
import spacy

nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # Remove special characters
    text = ' '.join([token.lemma_ for token in nlp(text) if not token.is_stop])  # Lemmatization
    return text

# Apply cleaning to the 'comment' column
df_exploded["cleaned_comment"] = df_exploded["comment"].apply(clean_text)

print(df_exploded[["comment", "cleaned_comment"]].head())  # Verify results

                                             comment  \
0  For emphasis: \n\n"Ghost prefers to use public...   
1  > The Ghost ransomware campaign highlights the...   
2  A joint security advisory published Feb. 19 by...   
3  I’m sure the new fbi director will handle this...   
4  The attacks have been ongoing this whole time....   

                                     cleaned_comment  
0  emphasis ghost prefer use publicly available c...  
1    Ghost ransomware campaign highlight persiste...  
2  joint security advisory publish Feb 19 FBI Cyb...  
3  m sure new fbi director handle competent way p...  
4  attack ongoing time not start stoppedwhat diff...  


In [ ]:
from bertopic import BERTopic

topic_model = BERTopic()
